In [1]:
import os
import sys
import numpy as np
import sklearn as skl
import time
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.model_selection import train_test_split

sys.path.append("../../../")
import CalibratedTimeseriesModels

from CalibratedTimeseriesModels.utils import *
from CalibratedTimeseriesModels.models.gnn import *
from CalibratedTimeseriesModels.evaluators import ExplicitEvaluator

%load_ext autoreload
%autoreload 2

In [2]:
# load data
X_orig = torch.load("../../../datasets/processed/openEI/X_openei_011_subset_multitask.pt")
Y_orig = torch.load("../../../datasets/processed/openEI/Y_openei_011_subset_multitask.pt")

X = X_orig[0,:,:24].unsqueeze(-1).float()
Y = Y_orig[0,:,:12].unsqueeze(-1).float()

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, train_size = 0.7/0.9,random_state=1)
X_batches, Y_batches = batch(X_train, Y_train)

### Diagonal Covariance Matrix

In [16]:
hidden_layers = [40, 40, 40]
model = GaussianNeuralNet(1, 24, hidden_layers, 1, 12)

In [17]:
train(model, X_batches, Y_batches, num_epochs=60)

epoch : 1, loss: -0.1029
epoch : 2, loss: -6.9972
epoch : 3, loss: -11.8197
epoch : 4, loss: -12.4267
epoch : 5, loss: -12.8769
epoch : 6, loss: -13.3783
epoch : 7, loss: -13.8912
epoch : 8, loss: -14.3649
epoch : 9, loss: -14.6143
epoch : 10, loss: -15.1315
epoch : 11, loss: -15.0475
epoch : 12, loss: -14.8431
epoch : 13, loss: -15.3840
epoch : 14, loss: -15.3275
epoch : 15, loss: -15.6596
epoch : 16, loss: -16.0817
epoch : 17, loss: -15.7227
epoch : 18, loss: -15.7463
epoch : 19, loss: -15.8284
epoch : 20, loss: -16.1419
epoch : 21, loss: -15.8840
epoch : 22, loss: -16.0603
epoch : 23, loss: -15.9389
epoch : 24, loss: -16.1572
epoch : 25, loss: -16.3103
epoch : 26, loss: -16.5619
epoch : 27, loss: -16.5702
epoch : 28, loss: -16.8463
epoch : 29, loss: -16.7645
epoch : 30, loss: -16.8436
epoch : 31, loss: -16.9051
epoch : 32, loss: -16.7386
epoch : 33, loss: -16.8406
epoch : 34, loss: -16.8964
epoch : 35, loss: -16.7397
epoch : 36, loss: -17.1003
epoch : 37, loss: -16.8603
epoch : 38, 

In [18]:
dtest = model(X_test)
mape(dtest, Y_test)

(tensor(0.0367, grad_fn=<MeanBackward0>),
 tensor([0.0260, 0.0288, 0.0293, 0.0312, 0.0336, 0.0418, 0.0429, 0.0382, 0.0410,
         0.0432, 0.0450, 0.0396], grad_fn=<MeanBackward1>),
 tensor([0.0229, 0.0250, 0.0265, 0.0292, 0.0314, 0.0355, 0.0355, 0.0322, 0.0367,
         0.0402, 0.0421, 0.0380], grad_fn=<StdBackward1>))

In [19]:
rmse(dtest, Y_test)

(tensor(0.0510, grad_fn=<PowBackward0>),
 tensor([0.0016, 0.0019, 0.0018, 0.0020, 0.0023, 0.0031, 0.0033, 0.0026, 0.0031,
         0.0032, 0.0035, 0.0029], grad_fn=<MeanBackward1>),
 tensor([0.0032, 0.0049, 0.0047, 0.0052, 0.0059, 0.0056, 0.0061, 0.0044, 0.0056,
         0.0062, 0.0070, 0.0054], grad_fn=<StdBackward1>))

In [20]:
nll(dtest, Y_test)

(tensor(-19.5743, grad_fn=<MeanBackward0>),
 tensor([-20.3771, -20.0503,  50.1044, -15.6146, -19.4976, -24.5063, -22.0410,
         -18.3323, -24.7329, -26.7688, -16.5699, -14.9878, -24.3020, -21.4480,
         -19.7465, -16.7804, -24.3132, -17.0713, -19.9366, -25.1299, -21.7302,
         -19.5093, -23.2317, -24.4029, -17.4278, -15.3351, -26.3236, -17.0145,
         -25.4478, -25.4584, -23.1114, -20.0582, -20.2174, -18.5640, -25.1007,
         -21.9937, -10.9783,  26.2981, -21.9779, -23.2254, -13.6814, -24.1219,
         -21.6421, -18.6018, -20.0147, -22.4739, -11.7606, -11.5997, -22.1841,
         -24.4418, -18.6160, -22.3303, -23.0140, -20.5605, -24.9485, -28.6246,
         -15.3084, -20.4541, -15.5021, -21.7190, -14.4454, -26.1499, -20.1200,
         -23.3968, -25.5928, -22.2167, -20.0023,  -5.6641, -19.7642, -23.6748,
         -18.8556,  -3.2985, -24.3281, -20.1020, -20.0012, -13.6152, -20.1795,
         -26.0380,  -3.1119, -22.4194, -17.4729, -23.1181, -24.8369, -27.3120,
        

### Full Covariance Matrix

In [21]:
hidden_layers = [40, 40, 40]
model = GaussianNeuralNet(1, 24, hidden_layers, 1, 12, covariance_type='full')

In [22]:
train(model, X_batches, Y_batches, num_epochs=60)

epoch : 1, loss: 0.7563
epoch : 2, loss: -4.2190
epoch : 3, loss: -5.4903
epoch : 4, loss: -3.7717
epoch : 5, loss: -7.2483
epoch : 6, loss: -8.5624
epoch : 7, loss: -9.5073
epoch : 8, loss: -5.2787
epoch : 9, loss: 3.0143
epoch : 10, loss: -3.5749
epoch : 11, loss: -5.9299
epoch : 12, loss: -6.9604
epoch : 13, loss: -8.0668
epoch : 14, loss: -9.8617
epoch : 15, loss: -10.1813
epoch : 16, loss: -10.7907
epoch : 17, loss: -11.1970
epoch : 18, loss: -11.1961
epoch : 19, loss: -11.5317
epoch : 20, loss: -11.4176
epoch : 21, loss: -11.6916
epoch : 22, loss: -11.7135
epoch : 23, loss: -12.1194
epoch : 24, loss: -10.8978
epoch : 25, loss: -9.2711
epoch : 26, loss: -12.5709
epoch : 27, loss: -12.3823
epoch : 28, loss: -12.5376
epoch : 29, loss: nan
epoch : 30, loss: nan
epoch : 31, loss: nan
epoch : 32, loss: nan
epoch : 33, loss: nan
epoch : 34, loss: nan
epoch : 35, loss: nan
epoch : 36, loss: nan
epoch : 37, loss: nan
epoch : 38, loss: nan
epoch : 39, loss: nan
epoch : 40, loss: nan
epoch 

In [23]:
dtest = model(X_test)
mape(dtest, Y_test)

(tensor(nan, grad_fn=<MeanBackward0>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        grad_fn=<MeanBackward1>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        grad_fn=<StdBackward1>))

In [24]:
rmse(dtest, Y_test)

(tensor(nan, grad_fn=<PowBackward0>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        grad_fn=<MeanBackward1>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        grad_fn=<StdBackward1>))

In [25]:
nll(dtest, Y_test)

(tensor(nan, grad_fn=<MeanBackward0>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

## Low Rank Covariance

In [26]:
hidden_layers = [40, 40, 40]
model = GaussianNeuralNet(1, 24, hidden_layers, 1, 12, covariance_type='low-rank',rank=2)

In [27]:
train(model, X_batches, Y_batches, num_epochs=60)

epoch : 1, loss: -3.1063
epoch : 2, loss: -10.3741
epoch : 3, loss: -13.1426
epoch : 4, loss: -13.9573
epoch : 5, loss: -15.1143
epoch : 6, loss: -15.5392
epoch : 7, loss: -16.3491
epoch : 8, loss: -16.2735
epoch : 9, loss: -16.6232
epoch : 10, loss: -17.3809
epoch : 11, loss: -17.5345
epoch : 12, loss: -17.5058
epoch : 13, loss: -17.8436
epoch : 14, loss: -18.9384
epoch : 15, loss: -18.8944
epoch : 16, loss: -19.0581
epoch : 17, loss: -18.9205
epoch : 18, loss: -18.8942
epoch : 19, loss: -19.5809
epoch : 20, loss: -19.2164
epoch : 21, loss: -19.2544
epoch : 22, loss: -19.8083
epoch : 23, loss: -20.0510
epoch : 24, loss: -20.0107
epoch : 25, loss: -19.8694
epoch : 26, loss: -20.0495
epoch : 27, loss: -19.5497
epoch : 28, loss: -20.2489
epoch : 29, loss: -20.0864
epoch : 30, loss: -19.4829
epoch : 31, loss: -20.6986
epoch : 32, loss: -20.9322
epoch : 33, loss: -20.8545
epoch : 34, loss: -20.9490
epoch : 35, loss: -20.9148
epoch : 36, loss: -21.2406
epoch : 37, loss: -20.3568
epoch : 38,

In [28]:
dtest = model(X_test)
mape(dtest, Y_test)

(tensor(0.0704, grad_fn=<MeanBackward0>),
 tensor([0.0425, 0.0565, 0.0779, 0.0767, 0.0670, 0.0730, 0.0723, 0.0661, 0.0783,
         0.0890, 0.0793, 0.0660], grad_fn=<MeanBackward1>),
 tensor([0.0390, 0.0458, 0.0601, 0.0615, 0.0585, 0.0618, 0.0592, 0.0518, 0.0590,
         0.0708, 0.0636, 0.0581], grad_fn=<StdBackward1>))

In [29]:
rmse(dtest, Y_test)

(tensor(0.0981, grad_fn=<PowBackward0>),
 tensor([0.0032, 0.0058, 0.0122, 0.0102, 0.0078, 0.0101, 0.0102, 0.0081, 0.0126,
         0.0171, 0.0110, 0.0072], grad_fn=<MeanBackward1>),
 tensor([0.0051, 0.0092, 0.0207, 0.0175, 0.0150, 0.0195, 0.0206, 0.0135, 0.0217,
         0.0311, 0.0160, 0.0117], grad_fn=<StdBackward1>))

In [30]:
nll(dtest, Y_test)

(tensor(-23.0435, grad_fn=<MeanBackward0>),
 tensor([-17.6379, -23.9581, -25.1549, -20.1730, -17.7164, -24.7863, -20.4652,
         -21.1271, -27.3322, -27.4133, -22.9483, -21.7543, -27.9186, -25.7621,
         -25.4281, -22.6882, -26.1279, -17.0008, -22.4901, -26.4073, -26.5508,
         -21.5318, -25.9927, -24.5449, -26.0411, -22.2509, -29.5128, -16.7363,
         -28.9746, -25.0010, -20.5155, -27.7893, -17.7024, -24.9991, -26.9579,
         -24.7814, -16.6304, -20.2253, -20.2465, -27.8986, -15.2050, -27.1029,
         -26.3821, -20.8268, -20.2792, -26.2254, -21.0821, -18.0084, -25.5994,
         -31.4710, -22.6845, -19.0172, -24.4441, -19.2032, -26.3612, -31.5482,
         -15.8553, -21.7509, -20.3203, -28.4922, -22.5282, -29.5870, -18.9504,
         -23.0070, -25.7244, -23.4984, -28.7966,  -6.2789, -25.3086, -28.8739,
         -17.6948, -17.6005, -27.0394, -24.5648, -21.8508, -21.0781, -22.8716,
         -27.1332,   2.8816, -25.1393, -24.0561, -25.8141, -29.9049, -26.4026,
        

In [33]:
dtest.sample((200,)).shape

torch.Size([200, 872, 12])